In [50]:
import os 
import json
import pandas as pd
import traceback

In [51]:
from langchain.chat_models import ChatOpenAI

In [52]:
from dotenv import load_dotenv
load_dotenv()

True

In [53]:
API_KEY = os.getenv("MCQGEN_API_KEY")

In [54]:
llm = ChatOpenAI(openai_api_key=API_KEY, model_name='gpt-3.5-turbo', temperature=0.5)

In [55]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [56]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    }
}

In [57]:
TEMPLATE1 = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""


In [58]:
prompt1 = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE1
)

In [59]:
chain1 = LLMChain(llm=llm, prompt=prompt1, output_key="quiz", verbose=True)

In [60]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [61]:
prompt2 = PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2
)

In [62]:
chain2 = LLMChain(llm=llm, prompt=prompt2, output_key="review", verbose=True)

In [63]:
combined_chain = SequentialChain(chains=[chain1, chain2], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [64]:
data_file_path = r'C:\Users\makin\Desktop\My workspace\GENAI\data.txt'
with open(data_file_path, 'r') as file:
    data = file.read()

In [65]:
dumps = json.dumps(RESPONSE_JSON)

In [66]:
TEXT = data
NUMBER = 5
SUBJECT = "Generative AI"
TONE = "simple"

In [67]:
# Tracking usage
with get_openai_callback() as cb:
    response = combined_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": dumps
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The academic discipline of artificial intelligence was established at a research workshop held at Dartmouth College in 1956 and has experienced several waves of advancement and optimism in the decades since.[20] Since its inception, researchers in the field have raised philosophical and ethical arguments about the nature of the human mind and the consequences of creating artificial beings with human-like intelligence; these issues have previously been explored by myth, fiction and philosophy since antiquity.[21] The concept of automated art dates back at least to the automata of ancient Greek civilization, where inventors such as Daedalus and Hero of Alexandria were described as having designed machines capable of writing text, generating sounds, and playing music.[22][23] The tradition of creative automatons has flourished throughout history, exemplified by Maillardet's automato

In [68]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2111
Prompt Tokens:1658
Completion Tokens:453
Total Cost:0.003393


In [71]:
QUIZ = response.get("quiz")
QUIZ = json.loads(QUIZ)

In [72]:
QUIZ

{'1': {'mcq': 'What was the groundbreaking 1950 paper by Alan Turing that posed fundamental questions about machine reasoning similar to human intelligence?',
  'options': {'a': 'Computing Machinery and Intelligence',
   'b': 'The Dartmouth Workshop',
   'c': 'Generative Adversarial Network',
   'd': 'Automated Art'},
  'correct': 'a'},
 '2': {'mcq': 'Which ancient Greek inventors were described as having designed machines capable of writing text, generating sounds, and playing music?',
  'options': {'a': 'Daedalus and Hero of Alexandria',
   'b': 'Harold Cohen',
   'c': 'Andrey Markov',
   'd': 'Alan Turing'},
  'correct': 'a'},
 '3': {'mcq': 'Which generative model in 2014 produced the first practical deep neural networks capable of learning generative models for complex data such as images?',
  'options': {'a': 'Variational Autoencoder',
   'b': 'GPT-1',
   'c': 'Markov Chains',
   'd': 'Generative Adversarial Network'},
  'correct': 'd'},
 '4': {'mcq': 'What AI model released by Me